## Connect to OGC API - Features service

The YIPEEO vector data is exposed as [OGC API
features](https://ogcapi.ogc.org/features/) accessible via
[https://features.dev.services.eodc.eu/].

Available YIPEEO data:
- yipeeo_yield_fl - Yield data at field level. - **protected**
- yipeeo_yield_nut2 - Yield data at NUTS2 scale. - public

The yipeeo_yield_fl data is protected and only accessible using an EODC account.
This tutorial demonstates the use of the OGC API - Features making use to the
[owslib](https://owslib.readthedocs.io/en/latest/index.html) Python client and
the [EODC SDK](https://pypi.org/project/eodc/) for authentication.

In [1]:
from rich.console import Console

# EODC SDK
from eodc.auth import DaskOIDC

# OWSLIB
from owslib.ogcapi.features import Features

# EODC OGC API URL
EODC_OGCAPI_URL = 'https://features.dev.services.eodc.eu/'

console = Console()

## Without Authentication

Without Authentication, we can still list all available feature collections, as
well as all features from collections which are not protected. But we are not
able to read features from the protected collection 'yipeeo_yield_fl'

In [2]:
# create eodc_ogcapi object without authentication header
eodc_ogcapi = Features(EODC_OGCAPI_URL)
feature_collections = eodc_ogcapi.feature_collections()
console.print(feature_collections)

[
    'adriatic_vessels',
    'Vlbg_EGMS_L3_E',
    'Vlbg_EGMS_L3_U',
    'STATISTIK_AUSTRIA_DSR',
    'yipeeo_yield_fl',
    'yipeeo_yield_nut2',
    'world_administrative_boundaries'
]

In [5]:
collection_id = 'yipeeo_yield_fl'

# this will fail with an '401 Authorization required' error code
items = eodc_ogcapi.collection_items(collection_id)
console.print(items['features'][0]['properties'])

RuntimeError: <html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>openresty</center>
<p><em>Powered by <a href="https://apisix.apache.org/">APISIX</a>.</em></p></body>
</html>


## With Authentication

To read features from 'yipeeo_yield_fl', we need to authenticate with our EODC
credentials. A password prompt will be opened automatically.

After sucessful authentication, the access token will be used as HTTP header for
all future requests using OWSLIB. 

In [6]:
username = "tobias.stachl@eodc.eu"
conn = DaskOIDC(username)

headers = {
    "Authorization": f"Bearer {conn.token['access_token']}"
}

# add HTTP headers to eodc_ogcapi object
eodc_ogcapi = Features(EODC_OGCAPI_URL, headers=headers)

In [7]:
collection_id = 'yipeeo_yield_fl'
items = eodc_ogcapi.collection_items(collection_id)
console.print(items['features'][0]['properties'])

{'crop_type': 'spring barley', 'country_code': 'cz ', 'yield': None, 'c_year': 2017, 'crop_id': 'C1320'}